<a href="https://colab.research.google.com/github/redadmiral/python-for-journalists/blob/main/exercises/solutions/ProbeklausurFahrradMUC_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datenanalyse

Unter der URL https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/e65e0e26-ce1f-4e58-9260-beccac196e75/download/rad_2022_15min_export_04_02_23_r.csv stellt die Stadt München einen Datensatz mit Messdaten der Fahrradzähler in der Stadt zur Verfügung.

Eine kurze Beschreibung des Datensatzes findest du hier: https://opendata.muenchen.de/dataset/daten-der-raddauerzaehlstellen-muenchen-jahreszahlen/resource/e65e0e26-ce1f-4e58-9260-beccac196e75

## 1.1 Exploration

Lade den Datensatz in eine Variable und beschreibe die Struktur des Datensatzes in eigenen Worten. Gehe dabei auf die Bedeutung der Spalten auf und führe kurz aus, was der Datensatz abbildet. Belege deine Aussagen soweit möglich indem du das Ergebnis mit `pandas` aus dem Datensatz selbst ziehst.

In [ ]:
import pandas as pd

bikes = pd.read_csv("https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/e65e0e26-ce1f-4e58-9260-beccac196e75/download/rad_2022_15min_export_04_02_23_r.csv")

In [ ]:
len(bikes)

In [ ]:
bikes["zaehlstelle"].unique()

In [ ]:
bikes["richtung_1"].sum() + bikes["richtung_2"].sum() == bikes["gesamt"].sum()

Der Datensatz zeigt wie viele Radfahrer\*innen in 15 Minuten-Intervallen eine der insgesamt 6 Messstationen in München passiert haben. Der Datensatz hat 210.240 Einträge, die sich auf die sechs Messstationen 'Arnulf', 'Kreuther', 'Olympia', 'Hirsch', 'Margareten', 'Erhardt' aufteilen.

Die Spalten uhrzeit_start und uhrzeit_ende bezeichnen Start und Ende eines Messintervalls, zaehlstelle den Namen der Messstation. richtung_1 und richtung_2 geben die Fahrtrichtung der Radfahrenden an, gesamt die Summe aus richtung_1 und richtung_2

## 1.2 Rush Hour

Was ist die maximale Zahl an Radfahrern (egal in welche Richtung), die eine Messstation in einem Messintervall erfasst hat?

In [ ]:
bikes["gesamt"].max()

## 1.3 Aktivste Station

Welche Station hat über das gesamte Jahr 2022 die meisten Fahrräder gemessen? Wie viele Fahrräder waren das?

In [ ]:
bikes_time = bikes.groupby("zaehlstelle")["gesamt"].sum()
bikes_time

## 1.4 Jahresüberblick

Erstelle ein Liniendiagramm mit Datawrapper aus dem die Gesamtzahl der Fahrräder nach Station und Tag ablesbar ist.

Gib der Grafik einen sinnvollen Titel und fülle alle Quellen aus. Kopiere den Link zur veröffentlichten (!) Grafik in das Feld unten und füge einen Screenshot der Grafik mit ein.

In [ ]:
bikes_station = bikes.groupby(["zaehlstelle", "datum"]).sum()
bikes_station = bikes_station.unstack()["gesamt"]
bikes_station.to_csv("bikesstation.csv")

#https://datawrapper.dwcdn.net/IRwYS/1/

## 1.5 Stonks

Der Graph sieht sehr zittrig und unübersichtlich aus. Wie kannst du den Datensatz aggregieren, um ihn besser lesbar zu machen?

Beschreibe deine Idee in eigenen Worten, du musst sie nicht implementieren, auch wenn das natürlich eine gute Übung ist.

In [ ]:
bikes_station_week = bikes
bikes_station_week["datum"] = pd.to_datetime(bikes["datum"])
bikes_station_week = bikes_station_week.set_index("datum")
bikes_station_week = bikes_station_week.groupby("zaehlstelle").resample(rule='W').mean(numeric_only=True)["gesamt"].unstack()
bikes_station_week.to_csv("bikesstationweek.csv")

In [ ]:
stations = pd.read_csv("https://opendata.muenchen.de/dataset/aca4bcb6-d0ff-4634-b5b9-8b5d133ab08e/resource/211e882d-fadd-468a-bf8a-0014ae65a393/download/radzaehlstellen.csv")
stations = stations.join(bikes.set_index("zaehlstelle"), on="zaehlstelle", lsuffix="_left")
stations = stations.groupby("zaehlstelle").mean()[["gesamt", "latitude", "longitude"]]
stations.to_csv("meanstations.csv")

#https://datawrapper.dwcdn.net/TFQq7/1/


<ipython-input-9-e9620f55b906>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  stations = stations.groupby("zaehlstelle").mean()[["gesamt", "latitude", "longitude"]]


# Scraper

Auf der Seite https://www.km.bayern.de/eltern/schulsuche.html?s=&t=9999&r=9999&o=9999&u=0&m=3&seite=1 stellt das bayerische Kultusministerium eine Suchfunktion bereit, um nach Schulen zu suchen.

Scrape die ersten drei Seiten der Suche und extrahiere alle Adressen der Schulen. Speichere diese als Strings ohne HTML-Fragmente in einer Liste ab.

Nutze für das Scraping die Bibliotheken `bs4` und `requests`.

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
base_url = "https://www.km.bayern.de"

result = []

def parse_line(soup):
  name = soup.find("a").text
  link = base_url + soup.find("a")["href"]
  address = soup.find("span", {"class": "Right"}).get_text(separator=", ")
  return address

for i in range(1, 4):
  print(f"Parsing page {i}")
  response = requests.get(f"https://www.km.bayern.de/eltern/schulsuche.html?s=&t=9999&r=9999&o=9999&u=0&m=3&seite={i}")
  soup = BeautifulSoup(response.content.decode(), "html.parser")

  school_list = soup.find("ul", {"class": "ListSchools"})

  schools = school_list.findAll("li")

  for line in schools:
    result.append(parse_line(line))

  sleep(1)

result